In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import fnmatch
import errno    
import glob
from riboraptor.sradb import SRAdb

Populating the interactive namespace from numpy and matplotlib


In [2]:
db = SRAdb('/staging/as/skchoudh/SRAmetadb.sqlite')


# List tables

In [3]:
sra_tables = db.list_tables()
sra_tables

['metaInfo',
 'submission',
 'study',
 'sample',
 'experiment',
 'run',
 'sra',
 'sra_ft',
 'sra_ft_content',
 'sra_ft_segments',
 'sra_ft_segdir',
 'col_desc',
 'fastq']

# List fields in a table

In [4]:
db.list_fields('study')

['study_ID',
 'study_alias',
 'study_accession',
 'study_title',
 'study_type',
 'study_abstract',
 'broker_name',
 'center_name',
 'center_project_name',
 'study_description',
 'related_studies',
 'primary_study',
 'sra_link',
 'study_url_link',
 'xref_link',
 'study_entrez_link',
 'ddbj_link',
 'ena_link',
 'study_attribute',
 'submission_accession',
 'sradb_updated']

# Describe columns in a table

In [5]:
db.desc_table('study')

cid	name	type	notnull	dflt_value	pk

0	study_ID	REAL	0	None	0
1	study_alias	TEXT	0	None	0
2	study_accession	TEXT	0	None	0
3	study_title	TEXT	0	None	0
4	study_type	TEXT	0	None	0
5	study_abstract	TEXT	0	None	0
6	broker_name	TEXT	0	None	0
7	center_name	TEXT	0	None	0
8	center_project_name	TEXT	0	None	0
9	study_description	TEXT	0	None	0
10	related_studies	TEXT	0	None	0
11	primary_study	TEXT	0	None	0
12	sra_link	TEXT	0	None	0
13	study_url_link	TEXT	0	None	0
14	xref_link	TEXT	0	None	0
15	study_entrez_link	TEXT	0	None	0
16	ddbj_link	TEXT	0	None	0
17	ena_link	TEXT	0	None	0
18	study_attribute	TEXT	0	None	0
19	submission_accession	TEXT	0	None	0
20	sradb_updated	TEXT	0	None	0


# Query

In [6]:
db.get_query('SELECT * FROM study LIMIT 3')

broker_name center_name                         center_project_name  \
0        None        KEIO      Bacillus subtilis subsp. natto BEST195   
1        None        KEIO  Bacillus subtilis subsp. subtilis str. 168   
2        None      UT-MGS          Integrative Transcriptome Analysis   

  ddbj_link ena_link primary_study related_studies sra_link  \
0      None     None          None            None     None   
1      None     None          None            None     None   
2      None     None          None            None     None   

         sradb_updated  study_ID                  ...                   \
0  2016-11-20 16:42:15       1.0                  ...                    
1  2017-09-21 20:23:35       2.0                  ...                    
2  2016-11-20 16:42:16       3.0                  ...                    

  study_accession study_alias study_attribute  \
0       DRP000001   DRP000001            None   
1       DRP000002   DRP000002            None   
2       DRP000003   DRP000003            None   

                                   study_description study_entrez_link  \
0                                               None              None   
1  Whole genome resequencing of B. subtilis subti...              None   
2  Although recent studies have revealed that the...              None   

                                         study_title               study_type  \
0  Bacillus subtilis subsp. natto BEST195 genome ...  Whole Genome Sequencing   
1  Model organism for prokaryotic cell differenti...  Whole Genome Sequencing   
2  Comprehensive identification and characterizat...   Transcriptome Analysis   

                study_url_link submission_accession  \
0                         None            DRA000001   
1                         None            DRA000002   
2  DBTSS: http://dbtss.hgc.jp/            DRA000003   

                              xref_link  
0  pubmed: 20398357 || pubmed: 25329997  
1                      pubmed: 20398357  
2                      pubmed: 20400770  

[3 rows x 21 columns]

# Number of rows in each table

In [7]:
db.get_table_counts()

count
metaInfo               2
sample           4199180
submission        961295
sra_ft_content   5242098
run              5339443
study             155648
fastq            5252155
sra              5242098
sra_ft_segdir        984
sra_ft_segments  2964125
col_desc             129
sra_ft           5242098
experiment       4708115

# Which study types are the most prominent?

In [8]:
db.get_query('SELECT study_type AS StudyType, count(*) AS Number FROM "study" GROUP BY study_type order by Number DESC')

Number                 StudyType
0    45967   Whole Genome Sequencing
1    38853                      None
2    38512                     Other
3    16149    Transcriptome Analysis
4    14415              Metagenomics
5      763       Population Genomics
6      607               Epigenetics
7      230          Exome Sequencing
8      110           Cancer Genomics
9       31   Pooled Clone Sequencing
10       9        Synthetic Genomics
11       1  Transcriptome Sequencing
12       1  Whole Genome Sequencing

# Which Sequencing Instruments are most common?

In [9]:
db.get_query('SELECT instrument_model AS "Instrument Model", count( * ) AS Experiments FROM "experiment" GROUP BY instrument_model order by Experiments DESC').sort_values(by=['Instrument Model'])

Experiments                     Instrument Model
21         8704                               454 GS
32          986                            454 GS 20
9         49413                           454 GS FLX
50           10                          454 GS FLX 
4        148483                  454 GS FLX Titanium
15        16702                          454 GS FLX+
13        23956                        454 GS Junior
53            1              AB 310 Genetic Analyzer
42          111             AB 3130 Genetic Analyzer
44           55           AB 3130xL Genetic Analyzer
43           71             AB 3500 Genetic Analyzer
49           14           AB 3500xL Genetic Analyzer
39          260             AB 3730 Genetic Analyzer
31         1168           AB 3730xL Genetic Analyzer
30         2111             AB 5500 Genetic Analyzer
24         5138           AB 5500xl Genetic Analyzer
40          203  AB 5500xl-W Genetic Analysis System
37          323               AB SOLiD 3 Plus System
19        11025                    AB SOLiD 4 System
41          153                  AB SOLiD 4hq System
51            3                   AB SOLiD PI System
34          501                      AB SOLiD System
35          483                  AB SOLiD System 2.0
29         2581                  AB SOLiD System 3.0
38          319                           BGISEQ-500
25         4276                    Complete Genomics
26         3937                    Helicos HeliScope
47           18                         HiSeq X Five
8         53240                          HiSeq X Ten
14        19075             Illumina Genome Analyzer
5        112444          Illumina Genome Analyzer II
7         63587         Illumina Genome Analyzer IIx
52            2        Illumina Genome Analyzer IIx 
27         3675                    Illumina HiScanSQ
16        14584                  Illumina HiSeq 1000
20         9759                  Illumina HiSeq 1500
1       1352215                  Illumina HiSeq 2000
3        572648                  Illumina HiSeq 2500
23         6082                  Illumina HiSeq 3000
12        29911                  Illumina HiSeq 4000
2        634091                       Illumina MiSeq
54            1                      Illumina MiSeq 
36          361                     Illumina MiniSeq
48           18                 Illumina NextSeq 500
46           41                               Ion S5
11        31574                      Ion Torrent PGM
22         8420                   Ion Torrent Proton
33          857                               MinION
6         89496                          NextSeq 500
28         3259                          NextSeq 550
18        12056                            PacBio RS
17        14435                         PacBio RS II
45           53                               Sequel
10        32933                          unspecified
0       1362293                                 None

# Which assays are most common?

In [11]:
db.get_query('SELECT library_strategy AS "Library Strategy", count( * ) AS Runs FROM "experiment" GROUP BY library_strategy order by Runs DESC')

Library Strategy     Runs
0                     None  1362293
1                      WGS  1092229
2                 AMPLICON   690441
3                  RNA-Seq   627617
4                    OTHER   314145
5                      WXS   237358
6                    CLONE    89555
7                 ChIP-Seq    82832
8                POOLCLONE    53738
9            Bisulfite-Seq    32335
10                   SELEX    25875
11               miRNA-Seq    22405
12                     WGA    21078
13                 RAD-Seq    10290
14        Targeted-Capture     8427
15                ATAC-seq     6060
16               ncRNA-Seq     5360
17                     EST     3601
18                 RIP-Seq     3264
19  DNase-Hypersensitivity     2868
20               MeDIP-Seq     2442
21               MNase-Seq     2328
22                 MRE-Seq     2073
23                 FL-cDNA     2010
24                  Tn-Seq     1886
25                     WCS     1809
26                 MBD-Seq     1690
27                CLONEEND      523
28               FAIRE-seq      477
29                    Hi-C      323
30                     CTS      282
31                   other      214
32     Synthetic-Long-Read      195
33               FINISHING       40
34                ChIA-PET       30
35              VALIDATION       22